ZIP PY



In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
zip_file = "ga-zip-code-latitude-and-longitude.csv"

zip_df = pd.read_csv(zip_file)

zip_df.head()


,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,30002,Avondale Estates,GA,33.772122,-84.264910,-5,1,33.772122
1,30003,Norcross,GA,33.960353,-84.037859,-5,1,33.960353
2,30004,Alpharetta,GA,34.119177,-84.302920,-5,1,34.119177
3,30005,Alpharetta,GA,34.080035,-84.219290,-5,1,34.080035
4,30006,Marietta,GA,33.912473,-84.557181,-5,1,33.912473


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Also, handle NaN values
# weather_df = weather_df.dropna()
# humidity = weather_df["Humidity (%)"].astype(float)

# # Store 'Latitude' and 'Longitude' into  locations. 
locations = zip_df[["Latitude", "Longitude"]].astype(float)

# locations

Create a humidity Heatmap layer
fig = gmaps.figure()
fig


# heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
#                                  dissipating=False, max_intensity=max(weather_df["Humidity (%)"]),
#                                  point_radius = 2)

# fig.add_layer(heat_layer)
# Save the figure
# plt.savefig("output_data/zipmap.png")
# fig

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Save the figure
plt.savefig("output_data/hotelmap.png")
# Display figure
fig